In [2]:
import numpy as np
# import pandas as pd
# import ampligraph

from ampligraph.datasets import load_from_csv


X = load_from_csv('../src/', 'recommend_dataset.csv', sep=',')
# X[:5]
X_desk = X[:48000]
X_card = X[48000:]

from ampligraph.evaluation import train_test_split_no_unseen 
data = {}
num_test = int(len(X_desk) * (20 / 100))
data['train'], data['test'] = train_test_split_no_unseen(X_desk, test_size=num_test, seed=0, allow_duplication=False) 
data['train'] = np.concatenate((data['train'], X_card), axis=0)

In [3]:
data['test'][:5]

array([['Odd Questline Warrior – #202 Legend (tayloreve) – Wild S93',
        'has_card', 'Reckless Flurry'],
       ['Holy Paladin – Nozdormu Day Quest (July 15, 2022)', 'has_card',
        'Knight of Anointment'],
       ['Hero Power Mage – #300 Legend (Clark) – Castle Nathria',
        'has_card', 'Fire Sale'],
       ['Kazakusan Druid – #12 Legend (Scorro) – Onyxia’s Lair',
        'has_card', 'Lunar Eclipse'],
       ['Skeleton Mage – #6 Legend (Meati) – Castle Nathria', 'has_card',
        'Flurry (Rank 1)']], dtype=object)

In [13]:
from ampligraph.latent_features import TransE, DistMult, ComplEx

modelTransE = TransE(batches_count=100, 
                seed=0, 
                epochs=500, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

positives_filter = X
modelTransE.fit(data['train'], early_stopping = False)

Average TransE Loss:   0.112030: 100%|██████████| 200/200 [09:58<00:00,  2.99s/epoch]


In [14]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
from ampligraph.evaluation import evaluate_performance

ranks = evaluate_performance(data['test'], 
                            model=modelTransE, 
                            filter_triples=positives_filter,   # Corruption strategy filter defined above 
                            use_default_protocol=True, # corrupt subj and obj separately while evaluating
                            verbose=True)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 9600/9600 [03:09<00:00, 50.57it/s]

MRR: 0.09


In [15]:
from ampligraph.discovery import query_topn
query_topn(modelTransE, top_n=5,
            head='XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End', relation='has_card', tail=None,
            ents_to_consider=None, rels_to_consider=None)

(array([['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
         'has_card', 'Gone Fishin’'],
        ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
         'has_card', 'Sira’kess Cultist'],
        ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
         'has_card', 'Dead Man’s Hand'],
        ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
         'has_card', 'Swiftscale Trickster'],
        ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
         'has_card', 'Crabatoa']], dtype='<U92'),
 array([-18.665962, -21.576368, -21.976175, -22.223473, -22.619543],
       dtype=float32))

In [6]:
from ampligraph.latent_features import TransE
modelTransE_all_data = TransE(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

modelTransE_all_data.fit(X, early_stopping = False)

Average TransE Loss:   0.103971: 100%|██████████| 200/200 [08:52<00:00,  2.66s/epoch]


In [7]:
from ampligraph.utils import save_model
example_name = 'recommend.pkl'
save_model(modelTransE_all_data, model_name_path = example_name)

In [14]:
from ampligraph.utils import restore_model
restored_model = restore_model(model_name_path = example_name)

In [15]:
from ampligraph.discovery import query_topn
query_topn(restored_model, top_n=5,
            head='XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End', relation='has_card', tail=None,
            ents_to_consider=None, rels_to_consider=None)[0]

array([['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
        'has_card', 'Gone Fishin’'],
       ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
        'has_card', 'Sira’kess Cultist'],
       ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
        'has_card', 'Za’qul'],
       ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
        'has_card', 'Ara’lon'],
       ['XL Secret Thief Rogue – #9 Legend (syoutotolo) – Knights of Hallow’s End',
        'has_card', 'Reconnaissance']], dtype='<U92')